In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import ast


In [2]:
df = pd.read_csv("/kaggle/input/recipenlg/full_dataset.csv")

print(df.head())

   Unnamed: 0                  title  \
0           0    No-Bake Nut Cookies   
1           1  Jewell Ball'S Chicken   
2           2            Creamy Corn   
3           3          Chicken Funny   
4           4   Reeses Cups(Candy)     

                                         ingredients  \
0  ["1 c. firmly packed brown sugar", "1/2 c. eva...   
1  ["1 small jar chipped beef, cut up", "4 boned ...   
2  ["2 (16 oz.) pkg. frozen corn", "1 (8 oz.) pkg...   
3  ["1 large whole chicken", "2 (10 1/2 oz.) cans...   
4  ["1 c. peanut butter", "3/4 c. graham cracker ...   

                                          directions  \
0  ["In a heavy 2-quart saucepan, mix brown sugar...   
1  ["Place chipped beef on bottom of baking dish....   
2  ["In a slow cooker, combine all ingredients. C...   
3  ["Boil and debone chicken.", "Put bite size pi...   
4  ["Combine first four ingredients and press in ...   

                                              link    source  \
0   www.cookbooks.com

In [3]:
print("Colonne:", df.columns.tolist())

Colonne: ['Unnamed: 0', 'title', 'ingredients', 'directions', 'link', 'source', 'NER']


In [4]:
rows_with_null = df.isnull().any(axis=1).sum()
print(f"There are {rows_with_null} rows with at least one null value.")


There are 1 rows with at least one null value.


In [5]:
print(f"In the dataset there are {len(df)} different receipts!")

In the dataset there are 2231142 different receipts!


In [6]:
ingredienti_unici = set()

for ingr_str in df['ingredients']:
    try:
        lista_ingredienti = ast.literal_eval(ingr_str)
        for ingrediente in lista_ingredienti:
            ingrediente_pulito = ingrediente.lower().strip()
            ingredienti_unici.add(ingrediente_pulito)
    except:
        continue

print(f"Total number of unique ingredients: {len(ingredienti_unici)}")


Total number of unique ingredients: 4472032
